## Perform enrichment test using phenotype ontology

Given a file of genes:

```
$ head data/rp-genes.tsv
NCBIGene:6295   SAG
NCBIGene:1258   CNGB1
NCBIGene:3614   IMPDH1
NCBIGene:26121  PRPF31
```

The example file here is derived from a Monarch query for all retinitis pigmentosa genes.

We want to test each class in HPO to see if it is enriched for genes in this set.

First we need to parse the gene Ids in the file

In [1]:
!pip install --upgrade --force-reinstall ontobio

  Using cached ontobio-2.8.3-py3-none-any.whl (310 kB)
  Using cached matplotlib-3.5.3-cp38-cp38-macosx_10_9_x86_64.whl (7.3 MB)
  Using cached pytest_logging-2015.11.4-py3-none-any.whl
  Using cached jsobject-0.10.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 32.7 MB/s eta 0:00:0000:0100:01
  Using cached PyYAML-6.0-cp38-cp38-macosx_10_9_x86_64.whl (192 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 21.3 MB/s eta 0:00:00
  Using cached pysolr-3.9.0-py2.py3-none-any.whl
  Using cached prefixcommons-0.1.12-py3-none-any.whl (29 kB)
  Using cached dacite-1.6.0-py3-none-any.whl (12 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached scipy-1.9.1-cp38-cp38-macosx_12_0_universal2.macosx_10_9_x86_64.whl (58.1 MB)
  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl (28 kB)
  Using cached yamldown-0.1.8-

In [2]:
## Parse ids from file
file = open("rp-genes.tsv", "r")
gene_ids = [row.split("\t")[0] for row in file]

## show first 10 IDs:
gene_ids[:10]

['NCBIGene:6295',
 'NCBIGene:1258',
 'NCBIGene:3614',
 'NCBIGene:26121',
 'NCBIGene:7275',
 'NCBIGene:55857',
 'NCBIGene:79797',
 'NCBIGene:10594',
 'NCBIGene:64218',
 'NCBIGene:7401']

In [3]:
## Create an ontology factory in order to fetch HPO
from ontobio.ontol_factory import OntologyFactory

ofactory = OntologyFactory()
ont = ofactory.create("hp")  
## Load HP. Note the first time this runs Jupyter will show '*' - be patient
ont


h:hp g:MultiDiGraph with 31649 nodes and 63349 edges

In [4]:
## Create an association factory to get gene-phenotype associations
from ontobio.config import session

from ontobio.assoc_factory import AssociationSetFactory
afactory = AssociationSetFactory()
## Load Associations from Monarch. Note the first time this runs Jupyter will show '*' - be patient
aset = afactory.create(ontology=ont, subject_category='gene', object_category='phenotype', taxon='NCBITaxon:9606')
aset

In [5]:
## Run enrichment tests using all classes in ontology
enr = aset.enrichment_test(subjects=gene_ids, threshold=0.00005, labels=True)

In [6]:
## Show first 20 results
print(enr)
for r in enr[:20]:
    print(r)
    print("{:8.3g} {} {:40s}".format(r['p'],r['c'],str(r['n'])))


[]


Given that the initial gene set is for retinitis pigmentosa genes, it's not surprising that enriched phenotype
terms are related to retinal degeneration

## Viewing Results

We can use different visualization options to see the enriched terms. First we will show a simple tree view


In [7]:
## Get all enriched class Ids
terms = [r['c'] for r in enr]

In [8]:
## Create a minimal slim of HPO consisting of enriched terms,
## with non-informative intermediate nodes removed
from ontobio.slimmer import get_minimal_subgraph
g = get_minimal_subgraph(ont.get_graph(), terms)

In [9]:
## Render as ascii-tree
from ontobio.graph_io import GraphRenderer
w = GraphRenderer.create('tree')
w.write(g, query_ids=terms)

ModuleNotFoundError: No module named 'ontobio.graph_io'

## visualization

Now we will show enriched terms in a graph using graphviz

In [ ]:
terms = [r['c'] for r in enr[:30]]
g = get_minimal_subgraph(ont.get_graph(), terms)
w = GraphRenderer.create('png')
w.outfile = "output/enr.png"
w.write(g, query_ids=terms)

## Image

![title](output/enr.png)